## Check for atypical (negative) step length

In [1]:
import glob
import os

import numpy as np
import pandas as pd

load_dir = os.path.join('..', '..', 'data', 'zeno', 'raw')

labels = [
    'Step Length (cm.)', 'Stride Length (cm.)', 'Stride Width (cm.)',
    'Stride Velocity (cm./sec.)', 'Absolute Step Length (cm.)',
    'Stride Time (sec.)', 'Stance %', 'Total D. Support %'
]

new_labels = [
    'step_length', 'stride_length', 'stride_width', 'stride_velocity',
    'absolute_step_length', 'stride_time', 'stance_percentage',
    'double_stance_percentage'
]

label_dict = {k: v for k, v in zip(labels, new_labels)}

# All files with .xlsx extension
file_paths = sorted(glob.glob(os.path.join(load_dir, '*.xlsx')))

# %% Read gait metrics from each Zeno file

list_l, list_r = [], []

In [2]:
has_neg_step = []

for file_path in file_paths:

    base_name = os.path.basename(file_path)  # File with extension
    file_name = os.path.splitext(base_name)[0]  # File with no extension

    df = pd.read_excel(file_path)

    # Locate the gait metric labels in the Excel file
    bool_array = df.applymap(
        lambda x: 'Step Time' in x if isinstance(x, str) else False).values

    row_gait, _ = np.argwhere(bool_array)[0]
    df.columns = df.iloc[row_gait, :]

    df_gait = df.iloc[row_gait + 1:]

    df_gait.columns.values[0] = 'Type'
    df_gait.columns.values[1] = 'Side'
    df_gait = df_gait.set_index(['Type', 'Side'])
    
    df_step_length = df_gait.filter(like='Step Length', axis=1)
    contains_neg = np.any(df_step_length < 0)

    has_neg_step.append(contains_neg)
    
    if np.any(contains_neg):
        
        break

In [6]:
file_name

'B_2014-12-22_P005_Post_tw_C'